In [17]:
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim


# Test verisini okuma
print("31")
test_df = pd.read_parquet('test-00000-of-00001.parquet')

# Train verisini okuma
train_df = pd.read_parquet('train-00000-of-00001.parquet')


31


In [18]:

print(train_df.tail())
print(test_df.tail())

                                                     text  label
119995  Pakistan's Musharraf Says Won't Quit as Army C...      0
119996  Renteria signing a top-shelf deal Red Sox gene...      1
119997  Saban not going to Dolphins yet The Miami Dolp...      1
119998  Today's NFL games PITTSBURGH at NY GIANTS Time...      1
119999  Nets get Carter from Raptors INDIANAPOLIS -- A...      1
                                                   text  label
7595  Around the world Ukrainian presidential candid...      0
7596  Void is filled with Clement With the supply of...      1
7597  Martinez leaves bitter Like Roger Clemens did ...      1
7598  5 of arthritis patients in Singapore take Bext...      2
7599  EBay gets into rentals EBay plans to buy the a...      2


In [19]:
# Eğitim seti
X_train = train_df['text']
y_train = train_df['label']

# Test seti
X_test = test_df['text']
y_test = test_df['label']


In [22]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data
def tokenize(texts, tokenizer, max_length=128):
    return tokenizer(
        texts.tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

train_tokens = tokenize(X_train, tokenizer)
test_tokens = tokenize(X_test, tokenizer)


In [23]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.tokens.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Assuming train_tokens, test_tokens, y_train, and y_test are defined
train_dataset = TextDataset(train_tokens, y_train.values)
test_dataset = TextDataset(test_tokens, y_test.values)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [24]:
from transformers import BertModel, BertTokenizer

bert = BertModel.from_pretrained('bert-base-uncased')

class TextCNN(nn.Module):
    def __init__(self, embed_dim, num_filters, filter_sizes, num_classes):
        super(TextCNN, self).__init__()
        self.bert = bert
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (size, embed_dim)) for size in filter_sizes
        ])
        self.fc = nn.Linear(num_filters * len(filter_sizes), num_classes)
        
    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            embeddings = self.bert(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        x = embeddings.unsqueeze(1)
        conv_outputs = [F.relu(conv(x)).squeeze(3) for conv in self.convs]
        pooled_outputs = [F.max_pool1d(conv_output, conv_output.size(2)).squeeze(2) for conv_output in conv_outputs]
        x = torch.cat(pooled_outputs, 1)
        x = self.fc(x)
        return x

# Model parametreleri
embed_dim = 768  # BERT hidden size
num_filters = 100
filter_sizes = [3, 4, 5]
num_classes = 4
model = TextCNN(embed_dim, num_filters, filter_sizes, num_classes)


In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)  # GPU/CPU'ya taşı
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)  # attention_mask eklendi
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


KeyboardInterrupt: 

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for batch in test_loader:
        inputs = batch['input_ids']
        labels = batch['labels']
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy: {100 * correct / total}%')
